<a href="https://colab.research.google.com/github/GoncalSousa20/NAS_IPL/blob/main/EfficientNetV2B3_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
# Define the paths to your three folders
train_path = '/content/drive/MyDrive/ProjetoCurso/train'
validation_path = '/content/drive/MyDrive/ProjetoCurso/validation'
test_path = '/content/drive/MyDrive/ProjetoCurso/test'

from keras.preprocessing.image import ImageDataGenerator
# Augment data
batch_size = 16
train_input_shape = (224, 224, 3)

train_datagen = ImageDataGenerator(rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/train',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

valid_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/validation',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                   )

test_generator = train_datagen.flow_from_directory(directory='/content/drive/MyDrive/ProjetoCurso/test',
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=1,
                                                    shuffle=False,
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID, "and", STEP_SIZE_TEST)

from keras.applications import EfficientNetV2B3
base_model = EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=train_input_shape)
for layer in base_model.layers:
    layer.trainable = False

from keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from keras.models import Model

n_classes = 11

# Add layers at the end
X = base_model.output
X = Flatten()(X)

# First dense block
X = Dense(512, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Second dense block
X = Dense(256, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Third dense block
X = Dense(128, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Fourth dense block
X = Dense(64, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Fifth dense block
X = Dense(32, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Final dense layer before output
X = Dense(16, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
# Optionally add dropout for regularization
# X = Dropout(0.5)(X)

# Output layer
output = Dense(n_classes, activation='softmax')(X)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

from keras.optimizers import Adam
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, cohen_kappa_score, hamming_loss, zero_one_loss, roc_auc_score,
    average_precision_score, brier_score_loss, classification_report
)
import pickle
import numpy as np
import tensorflow as tf
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
# Additional custom metrics
def fbeta_score(precision, recall, beta=1.0):
    if precision + recall == 0:
        return 0.0
    return (1 + beta*2) * (precision * recall) / (beta*2 * precision + recall)

def g_mean(precision, recall):
    return np.sqrt(precision * recall)

# Example usage
dictionary_metrics = {
    'train_acc': [],
    'train_loss': [],
    'precision': [],
    'recall': [],
    'val_loss': [],
    'test_acc': [],
    'test_loss': [],
    'val_accuracy': [],
    'f1_score': [],
    'mcc': [],
    'cohen_kappa': [],
    'hamming_loss': [],
    'zero_one_loss': [],
    'auc_roc': [],
    'brier_score': [],
    'mAP': [],
    'f1_beta': [],
    'g_mean': []
}

# Function to get predictions and labels in batches
def get_predictions_and_labels(generator, model, steps):
    y_true = []
    y_pred = []
    y_pred_prob = []

    for i in range(steps):
        (X, y) = next(generator)
        y_pred_prob_batch = model.predict(X)
        y_pred_batch = np.argmax(y_pred_prob_batch, axis=1)

        y_pred.append(y_pred_batch)
        y_pred_prob.append(y_pred_prob_batch)
        y_true.append(y)

    # Flatten the lists
    y_pred = np.concatenate(y_pred, axis=0)
    y_pred_prob = np.concatenate(y_pred_prob, axis=0)
    y_true = np.concatenate(y_true, axis=0)

    # Update Truth vector based on argmax if y is one-hot encoded
    if y_true.ndim > 1 and y_true.shape[1] > 1:
        y_true = np.argmax(y_true, axis=1)

    # Convert to numpy arrays
    y_pred = np.asarray(y_pred).ravel()
    y_pred_prob = np.asarray(y_pred_prob)

    return y_true, y_pred,y_pred_prob

Mounted at /content/drive
Found 3862 images belonging to 11 classes.
Found 216 images belonging to 11 classes.
Found 219 images belonging to 11 classes.
Total number of batches = 241 and 13 and 219
52606240/52606240 [==============================] - 0s 0us/step


In [ ]:
for epoch in range(30):
  print(f"Starting Epoch {epoch+1}")
  history = model.fit(
      train_generator,
      validation_data=valid_generator,
      epochs=1
  )
  print(history.history)
  print(f"Finished Epoch {epoch+1}")

  # Extract validation labels
  val_labels = []
  for i in range(len(valid_generator)):
      _, batch_labels = next(valid_generator)
      val_labels.extend(batch_labels)

  val_labels = np.array(val_labels)
  dictionary_metrics['train_acc'].append(history.history["accuracy"])
  dictionary_metrics['train_loss'].append(history.history["loss"])

  # Calculate metrics for validation and test sets
  val_loss, val_accuracy = model.evaluate(valid_generator)
  dictionary_metrics['val_accuracy'].append(val_accuracy)
  dictionary_metrics['val_loss'].append(val_loss)
  test_loss, test_acc = model.evaluate(test_generator)
  dictionary_metrics['test_loss'].append(test_loss)
  dictionary_metrics['test_acc'].append(test_acc)

  test_labels, test_pred_labels, test_pred_prob = get_predictions_and_labels(test_generator, model, steps=STEP_SIZE_TEST)

  num_classes = len(np.unique(test_labels))

  test_classification_report = classification_report(test_labels, test_pred_labels, output_dict=True)
  test_recall_list = []
  test_precision_list = []
  test_f1_score_list = []
  test_f1_beta_list = []
  test_g_mean_list = []

  for class_label in range(num_classes):
      recall = test_classification_report[str(class_label)]['recall']
      precision = test_classification_report[str(class_label)]['precision']
      f1 = test_classification_report[str(class_label)]['f1-score']
      test_recall_list.append(recall)
      test_precision_list.append(precision)
      test_f1_score_list.append(f1)
      test_f1_beta_list.append(fbeta_score(precision, recall, beta=2.0))  # Using beta=2 for F1-beta score
      test_g_mean_list.append(g_mean(precision, recall))

  dictionary_metrics['recall'].append(test_recall_list)
  dictionary_metrics['precision'].append(test_precision_list)
  dictionary_metrics['f1_score'].append(test_f1_score_list)
  dictionary_metrics['f1_beta'].append(test_f1_beta_list)
  dictionary_metrics['g_mean'].append(test_g_mean_list)
  dictionary_metrics['mcc'].append(matthews_corrcoef(test_labels, test_pred_labels))
  dictionary_metrics['cohen_kappa'].append(cohen_kappa_score(test_labels, test_pred_labels))
  dictionary_metrics['hamming_loss'].append(hamming_loss(test_labels, test_pred_labels))
  dictionary_metrics['zero_one_loss'].append(zero_one_loss(test_labels, test_pred_labels, normalize=True))

  try:
      test_roc_auc = roc_auc_score(test_labels, test_pred_prob, multi_class='ovr')
      dictionary_metrics['auc_roc'].append(test_roc_auc)
  except ValueError:
      print("Error in calculating ROC AUC for test set")

  try:
      avg_precision = []
      for i in range(num_classes):
          avg_precision.append(average_precision_score((test_labels == i).astype(int), test_pred_prob[:, i]))
      avg_precision = np.mean(avg_precision)
      dictionary_metrics['mAP'].append(avg_precision)
  except ValueError as e:
      print(f"Error in calculating Average Precision: {e}")

  # Calculate Brier score for each class and average them
  brier_scores = []
  for i in range(num_classes):
      true_binary = (test_labels == i).astype(float)
      probas = test_pred_prob[:, i]
      brier_score = brier_score_loss(true_binary, probas)
      brier_scores.append(brier_score)

  average_brier_score = np.mean(brier_scores)
  dictionary_metrics['brier_score'].append(average_brier_score)

with open('/content/drive/MyDrive/ProjetoCurso/metrics_EfficientNetV2B3.pkl', 'wb') as fp:
    pickle.dump(dictionary_metrics,fp)

Starting Epoch 1
242/242 [==============================] - 1758s 7s/step - loss: 2.5138 - accuracy: 0.1233 - val_loss: 2.3253 - val_accuracy: 0.2037
{'loss': [2.5137717723846436], 'accuracy': [0.12325219810009003], 'val_loss': [2.325259208679199], 'val_accuracy': [0.20370370149612427]}
Finished Epoch 1
1/1 [==============================] - 0s 49ms/step
Starting Epoch 2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 114s 472ms/step - loss: 2.3356 - accuracy: 0.1756 - val_loss: 2.3061 - val_accuracy: 0.2037
{'loss': [2.335588216781616], 'accuracy': [0.1755567044019699], 'val_loss': [2.3060903549194336], 'val_accuracy': [0.20370370149612427]}
Finished Epoch 2
1/1 [==============================] - 0s 31ms/step
Starting Epoch 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 106s 437ms/step - loss: 2.3159 - accuracy: 0.1869 - val_loss: 2.2949 - val_accuracy: 0.2037
{'loss': [2.315894603729248], 'accuracy': [0.18694975972175598], 'val_loss': [2.2948882579803467], 'val_accuracy': [0.20370370149612427]}
Finished Epoch 3
1/1 [==============================] - 0s 31ms/step
Starting Epoch 4


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 106s 437ms/step - loss: 2.1862 - accuracy: 0.2206 - val_loss: 2.1972 - val_accuracy: 0.1944
{'loss': [2.1862335205078125], 'accuracy': [0.2206110805273056], 'val_loss': [2.1972336769104004], 'val_accuracy': [0.1944444477558136]}
Finished Epoch 29
1/1 [==============================] - 0s 32ms/step
Starting Epoch 30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


242/242 [==============================] - 105s 435ms/step - loss: 2.1913 - accuracy: 0.2253 - val_loss: 2.1830 - val_accuracy: 0.2037
{'loss': [2.1913161277770996], 'accuracy': [0.22527188062667847], 'val_loss': [2.183004140853882], 'val_accuracy': [0.20370370149612427]}
Finished Epoch 30
1/1 [==============================] - 0s 45ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

# Function to print the metrics in an organized way
def print_metrics(metrics):
    print("Metrics:")
    print("--------")
    for key, value in metrics.items():
        if isinstance(value, list):  # Check if the value is a list (e.g., confusion matrix)
            print(f"{key}:")
            for row in value:
                print(f"  {row}")
        else:
            print(f"{key}: {value}")

# Reading the metrics from the pickle file
try:
    with open('/content/drive/MyDrive/ProjetoCurso/metrics_EfficientNetV2B3.pkl', 'rb') as fp:
        loaded_metrics = pickle.load(fp)
    print("Metrics successfully loaded from metrics.pkl")
    print_metrics(loaded_metrics)
except EOFError:
    print("Error: Ran out of input - the file may be empty or improperly written.")
except FileNotFoundError:
    print("Error: The file does not exist.")
except pickle.UnpicklingError:
    print("Error: The file contents are not a valid pickle.")


Metrics successfully loaded from metrics.pkl
Metrics:
--------
train_acc:
  [0.12325219810009003]
  [0.1755567044019699]
  [0.18694975972175598]
  [0.19368202984333038]
  [0.19937856495380402]
  [0.20274469256401062]
  [0.197048157453537]
  [0.20067322254180908]
  [0.2102537602186203]
  [0.20921802520751953]
  [0.2099948227405548]
  [0.20921802520751953]
  [0.21439668536186218]
  [0.21569135785102844]
  [0.21154841780662537]
  [0.20818229019641876]
  [0.22009322047233582]
  [0.21879854798316956]
  [0.2099948227405548]
  [0.21931642293930054]
  [0.20895908772945404]
  [0.2136198878288269]
  [0.217244952917099]
  [0.2208700180053711]
  [0.22527188062667847]
  [0.22501294314861298]
  [0.22527188062667847]
  [0.21517348289489746]
  [0.2206110805273056]
  [0.22527188062667847]
train_loss:
  [2.5137717723846436]
  [2.335588216781616]
  [2.315894603729248]
  [2.3077640533447266]
  [2.2976043224334717]
  [2.290296792984009]
  [2.284392833709717]
  [2.278409242630005]
  [2.2592828273773193]
  [

## Training graph

In [ ]:
# Merge history1 and history2
history = {}
history['loss'] = history1.history['loss'] + history2.history['loss']
history['acc'] = history1.history['acc'] + history2.history['acc']
history['val_loss'] = history1.history['val_loss'] + history2.history['val_loss']
history['val_acc'] = history1.history['val_acc'] + history2.history['val_acc']
history['lr'] = history1.history['lr'] + history2.history['lr']

In [ ]:
# Plot the training graph
def plot_training(history):
    acc = history['acc']
    val_acc = history['val_acc']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(15,5))

    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')

    plt.show()

plot_training(history)

## Evaluate performance

In [ ]:
# Prediction accuracy on train data
score = model.evaluate_generator(train_generator, verbose=1)
print("Prediction accuracy on train data =", score[1])

In [ ]:
# Prediction accuracy on CV data
score = model.evaluate_generator(valid_generator, verbose=1)
print("Prediction accuracy on CV data =", score[1])

In [ ]:
# Classification report and confusion matrix
from sklearn.metrics import *
import seaborn as sns

tick_labels = artists_top_name.tolist()

def showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID):
    # Loop on each generator batch and predict
    y_pred, y_true = [], []
    for i in range(STEP_SIZE_VALID):
        (X,y) = next(valid_generator)
        y_pred.append(model.predict(X))
        y_true.append(y)

    # Create a flat list for y_true and y_pred
    y_pred = [subresult for result in y_pred for subresult in result]
    y_true = [subresult for result in y_true for subresult in result]

    # Update Truth vector based on argmax
    y_true = np.argmax(y_true, axis=1)
    y_true = np.asarray(y_true).ravel()

    # Update Prediction vector based on argmax
    y_pred = np.argmax(y_pred, axis=1)
    y_pred = np.asarray(y_pred).ravel()

    # Confusion Matrix
    fig, ax = plt.subplots(figsize=(10,10))
    conf_matrix = confusion_matrix(y_true, y_pred, labels=np.arange(n_classes))
    conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
    sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False,
                cmap=plt.cm.jet, xticklabels=tick_labels, yticklabels=tick_labels,
                ax=ax)
    ax.set_ylabel('Actual')
    ax.set_xlabel('Predicted')
    ax.set_title('Confusion Matrix')
    plt.show()

    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=np.arange(n_classes), target_names=artists_top_name.tolist()))

showClassficationReport_Generator(model, valid_generator, STEP_SIZE_VALID)

# Evaluate performance by predicting on random images from dataset

In [ ]:
# Prediction
from keras.preprocessing import *

n = 5
fig, axes = plt.subplots(1, n, figsize=(25,10))

for i in range(n):
    random_artist = random.choice(artists_top_name)
    random_image = random.choice(os.listdir(os.path.join(images_dir, random_artist)))
    random_image_file = os.path.join(images_dir, random_artist, random_image)

    # Original image

    test_image = image.load_img(random_image_file, target_size=(train_input_shape[0:2]))

    # Predict artist
    test_image = image.img_to_array(test_image)
    test_image /= 255.
    test_image = np.expand_dims(test_image, axis=0)

    prediction = model.predict(test_image)
    prediction_probability = np.amax(prediction)
    prediction_idx = np.argmax(prediction)

    labels = train_generator.class_indices
    labels = dict((v,k) for k,v in labels.items())

    #print("Actual artist =", random_artist.replace('_', ' '))
    #print("Predicted artist =", labels[prediction_idx].replace('_', ' '))
    #print("Prediction probability =", prediction_probability*100, "%")

    title = "Actual artist = {}\nPredicted artist = {}\nPrediction probability = {:.2f} %" \
                .format(random_artist.replace('_', ' '), labels[prediction_idx].replace('_', ' '),
                        prediction_probability*100)

    # Print image
    axes[i].imshow(plt.imread(random_image_file))
    axes[i].set_title(title)
    axes[i].axis('off')

plt.show()